# Phi-4 Mini ONNX Párhuzamos Funkcióhívás Oktatóanyag

Ez a jegyzet bemutatja, hogyan használható a Phi-4 Mini az ONNX Runtime GenAI-val párhuzamos funkcióhívásokhoz. A funkcióhívás lehetővé teszi, hogy a modell intelligensen külső eszközöket és API-kat hívjon meg a felhasználói kérések alapján.

## Áttekintés

Ebben az oktatóanyagban megtanulhatod:
- Hogyan állítsd be a Phi-4 Mini-t az ONNX Runtime GenAI-val
- Hogyan definiálj funkciósémákat repülőjegyek és szállodák foglalásához
- Hogyan használd a Lark nyelvtant irányított generáláshoz strukturált kimenet érdekében
- Hogyan hajts végre párhuzamos funkcióhívásokat összetett utazási foglalási forgatókönyvekhez

## Előfeltételek

Mielőtt futtatnád ezt a jegyzetet, győződj meg róla, hogy:
- Letöltötted a Phi-4 Mini ONNX modellt
- Telepítetted az `onnxruntime-genai` csomagot
- Alapvető ismeretekkel rendelkezel a funkcióhívás fogalmáról


## 1. lépés: Szükséges könyvtárak importálása

Először importáljuk a szükséges könyvtárakat a függvényhívás megvalósításához.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## 2. lépés: Modell beállítása és konfigurálása

Most beállítjuk a Phi-4 Mini ONNX modellt. Ügyelj arra, hogy a megadott útvonalat cseréld ki a saját modellkönyvtárad elérési útjára.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## 3. lépés: A generálási paraméterek beállítása

Állítsd be a generálási paramétereket, hogy szabályozd a modell kimeneti viselkedését. Ezek a beállítások biztosítják a determinisztikus és célzott válaszokat a függvényhívásokhoz.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## 4. lépés: Elérhető funkciók meghatározása

Itt határozzuk meg azokat a funkciókat, amelyeket AI asszisztensünk használhat. Ebben a példában két utazással kapcsolatos funkciót definiálunk:
1. **booking_flight_tickets**: Repülőjegyek foglalása repülőterek között
2. **booking_hotels**: Szállodai szállásfoglalás

A funkciódefiníciók az OpenAI funkcióhívási séma formátumát követik.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## 5. lépés: Nyelvtan-generáló segédfüggvények

Ezek a segédfüggvények a függvénydefinícióinkat Lark nyelvtan formátumba alakítják, amelyet az ONNX Runtime GenAI használ az irányított generáláshoz. Ez biztosítja, hogy a modell érvényes függvényhívásokat adjon vissza a megfelelő JSON formátumban.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## 6. lépés: A nyelvtan generálás tesztelése

Teszteljük a nyelvtan generáló funkcióinkat, hogy lássuk, hogyan alakítják át az eszközdefinícióinkat a megfelelő formátumba.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## 7. lépés: Rendszerprompt és generátor előkészítése

Most elkészítjük azt a rendszerpromptot, amely tájékoztatja a modellt az elérhető eszközökről, és beállítjuk a generátort az irányított generálási paraméterekkel.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## 8. lépés: Generátor inicializálása irányított generálással

Most létrehozzuk a generátort az általunk beállított paraméterekkel, és alkalmazzuk a Lark nyelvtant az irányított generáláshoz.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## 9. lépés: Párhuzamos függvényhívások tesztelése

Most teszteljük a beállításunkat egy összetett utazási foglalási helyzetben, amely több függvény hívását igényli.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Funkcióhívások generálása

A modell most strukturált funkcióhívásokat fog generálni a felhasználói kérés alapján. Az irányított generálásnak köszönhetően a kimenet érvényes JSON formátumban lesz, amely közvetlenül végrehajtható.

**Várható kimenet**: A modellnek a következő funkcióhívásokat kell generálnia:
1. `booking_flight_tickets` Peking (PEK) és Párizs (CDG) közötti részletekkel
2. `booking_hotels` Párizsi szállás részletekkel

Futtassa az alábbi cellát az élő generálás megtekintéséhez:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Következtetés

🎉 **Gratulálunk!** Sikeresen megvalósítottad a párhuzamos függvényhívást Phi-4 Mini segítségével az ONNX Runtime GenAI használatával.

### Amit megtanultál:

1. **Modell beállítása**: Hogyan konfiguráld a Phi-4 Mini modellt az ONNX Runtime GenAI-val
2. **Függvények definiálása**: Hogyan határozd meg a függvények sémáit AI függvényhívásokhoz
3. **Irányított generálás**: Hogyan használd a Lark grammatikát strukturált kimenet generálásához
4. **Párhuzamos függvényhívások**: Hogyan kezeld az összetett kéréseket, amelyek több függvényhívást igényelnek

### Fő előnyök:

- ✅ **Strukturált kimenet**: Az irányított generálás biztosítja az érvényes JSON függvényhívásokat
- ✅ **Párhuzamos feldolgozás**: Több függvényhívás kezelése egyetlen kérésben
- ✅ **Magas teljesítmény**: Az ONNX Runtime optimalizált következtetést biztosít
- ✅ **Rugalmas séma**: Könnyen hozzáadható vagy módosítható függvénydefiníciók

### Források:

- [Phi-4 Mini Dokumentáció](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Dokumentáció](https://onnxruntime.ai/docs/genai/)
- [Függvényhívási legjobb gyakorlatok](https://platform.openai.com/docs/guides/function-calling)



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális, emberi fordítást igénybe venni. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
